# Grid Topology Reconstruction

#### Author: Bryan Murray
#### Last revision: March 23, 2023
The relationship between the electrical connectivity and the physical location of distribution grid components is not always obvious. Many utilites maintain separate databases for control of the grid's electrical operations and mapping of the wires, transformers, and meters in the real world. There can be very good reasons for such data management, including privacy and security concerns. Nevertheless, this practice impedes independent research in, for example, how distributed energy resources (DERs) can be best deployed on a particular feeder.

I'm interested in maximizing local, renewable power generally, but in many areas, the grid has technical constraints that limit how much DERs can be installed. The **hosting capacity analysis** (HCA) is a procedure to quantify just how much generation (or energy storage, or shiftable load) can be installed at each node in a grid before violating thermal limits of the lines, voltage fluction, voltage droop, or other power quality metrics. 

In this analysis, I'm specifically interested in the hosting capacity of a section of Southern California Edison's medium-voltage grid in Goleta, CA. Unfortunately, the only pulbically available data is a spaghetti bowl of lines segments, with no indication of their electrical connectivity. If you've read this far, you probably have some intuition about how these lines might be connected, so let's try to semi-automate a procedure for converting the lattitude/longitude coordinates into an electrical connectivity graph.

In [1]:
import numpy as np
from numpy.random import default_rng
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import geopandas
import folium
from shapely.geometry import Point, LineString
from geographiclib.geodesic import Geodesic
import pickle

# For brevity, much of the code has been abstracted into custom function, 
# available as part of the the MVProfessor repository: https://github.com/brmurray/mvprofessor
from mvprofessor.config import int_data_dir, maps_dir
import mvprofessor.custom_funcs as mvpf

# *****************************
# Layer 0: Points of Interest 
# *****************************
poi = pd.read_pickle(int_data_dir/'PoI_Professor.pkl')

# *****************************
# Layer 1: Professor feeder linstrings
# *****************************
gdf = pd.read_pickle(int_data_dir/'professor.pkl')
gdf = gdf[gdf['SHAPE__Length'] > 30] #with cutoff=20, len(gdf)=197

# Create other layers: linestring endpoints (pts) and combined buffers (blobs)
# *****************************
# Layer 2: LineSring endpoints
# *****************************
pts = mvpf.get_endpoints(gdf)


# First Map, just to show spaghetti line segments and places of interest
# Step 0: Note some important Places of Interest
m = poi.explore(color='green',marker_type='marker', show=False,
                name="Step 0: Note Points of Interest")

# Step 1: Show DRPEP Line Segments
gdf.explore(m=m,color='blue',legend=False, show=True,
            name="Step 1: DRPEP Line Sections",style_kwds=dict(weight=2))

# Step 2: Show Endpoints
pts.explore(m=m,color='blue',marker_kwds=dict(radius=2),show=False,
               name="Step 2: LineString endpoints")

folium.LayerControl().add_to(m)
m.save(maps_dir / 'Map_Steps_Intermediate.jpg')
display(m)

It doesn't take too much imagination to see a MV feeder here, but we still do not have a nicely defined set of buses and lines.

My solution was to buffer the endpoints of each linesegment, and combine the resulting circles into "blobs", which are irregular polygons containing 1 or more wire endpoints. These will become the electrical nodes, connected by the various line segments we see above.

In [2]:
# *****************************
# Layer 3: Blobs
# *****************************
blobs = mvpf.make_blobs(pts,7)
blobs['powered'] = 0

To connect the blobs, I wrote a function to recursively trace the wires from a given node to find neighbors, and collect all these into a Graph database (via NetworkX). The algorithm is inspired by a Depth-First Search (DFS) tree traversal, in which a "frontier" of neighboring nodes is explored until the terminal nodes are found. 


In [3]:
# Define a shapely.Point as the desired starting point
# In this case, we want to start at the Isla Vista Substation  
isla_vista = poi.loc[poi['PoI']=='isla_vista_ss']['geometry']
isla_vista = isla_vista.iloc[0]

# Run the tree builder algorithm
G = mvpf.tree_builder(gdf,blobs,isla_vista)

In [4]:
#Make a dataframe of the electrical nodes (enodes), and flag them by their subgraph
enodes = mvpf.make_enodes(G)

In [5]:
#%% Intermediate map to identify nodes that should be manually connected
# Step 0: Note some important Places of Interest
m = poi.explore(color='green',marker_type='marker', show=False,
                name="Step 0: Note Points of Interest")

# Step 1: Show DRPEP Line Segments
#gdf.explore(m=m,name="Step 1: DRPEP Line Sections",style_kwds=dict(weight=3))
# gdf.explore(m=m,column='randc',cmap='gist_rainbow',legend=False,
#             name="Step 1: DRPEP Line Sections",style_kwds=dict(weight=3))
gdf.explore(m=m,color='blue',legend=False, show=True,
            name="Step 1: DRPEP Line Sections",style_kwds=dict(weight=2))


# Step 2: Find Endpoints
pts.explore(m=m,color='blue',marker_kwds=dict(radius=2),show=False,
               name="Step 2: LineString endpoints")


# Step 3: Blobs
blobs.explore(m=m,color='green',marker_type='circle',
              style_kwds=dict(fillOpacity=0.2),show=False,
              name="Step 3: Combine very-near endpoints into 'blobs'")

# Step 4: Electrical nodes with colors
enodes.explore(m=m,column='randc',cmap='prism',marker_kwds=dict(radius=4),
                legend=False,style_kwds=dict(fillOpacity=1.0),show=True,
                name="Step 4: Infer Electrical Nodes (colored by sub-network)")

folium.LayerControl().add_to(m)
display(m)

Our electrical nodes are defined, and we can see that they following our intuition about how the lines connect. However, the "blobs" don't always connect neighboring nodes. Fortunately, it's relatively simple to "contract" the network using NetworkX.

In [6]:
# Define pairs of the nodes to combine
# The first entry in the tuple should be the "upstream" node, to be retained
# i.e. the first entry "absorbs" the second entry
pairs = [(50,47),(75,76),(85,84),(85,87),(125,126)]
for p in pairs:
    G=nx.contracted_nodes(G,p[0],p[1])

# Remake the enodes layer, because some nodes have been combined
enodes = mvpf.make_enodes(G)

Our keenest interest regards the bit of the feeder around Goleta City Hall (between Hollister Road and S. Los Carneros Road), which happens to be subgraph 0. NetworkX again makes it easy to extract the subgraph of these nodes (and edges).

In [7]:
# Focus on the subgraph around Goleta City Hall/ Karl Storz ("CHKS")
# S is a list of subgraphs (nx.graph objects)
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

sx = 0 # subgraph of interest. For City Hall/Karl Storz, sx=0

# *****************************
# Layer 5a: Enodes of City Hall/Karl Storz
# *****************************
chk_enodes = enodes[enodes.subgraph==sx]
chk_enodes.to_pickle(int_data_dir / 'CHKS_nodes.pkl')

# Edges of the subgraph
sedge_df = nx.to_pandas_edgelist(S[sx])

# *****************************
# Layer 5b: DRPEP LineStrings of City Hall/Karl Storz
# *****************************
enodes_chks = enodes[enodes.subgraph==sx]
chks_lines = geopandas.GeoDataFrame(sedge_df,geometry=sedge_df['geometry'],crs="EPSG:2955")
chks_lines = chks_lines.drop('branch',axis=1)
chks_lines.to_pickle(int_data_dir / 'CHKS_lines.pkl')

# *****************************
# Layer 5c: Direct lines connecting enodes of City Hall/Karl Storz
# *****************************
directlines = []
p = nx.get_node_attributes(S[sx],'pos')
for nodes in S[sx].edges:
    nor1, east1 = p[nodes[0]][0], p[nodes[0]][1]
    nor2, east2 = p[nodes[1]][0], p[nodes[1]][1]
    directlines.append(LineString([[nor1,east1],[nor2,east2]]))
dl = geopandas.GeoSeries(directlines,crs="EPSG:2955")
    
chks_direct = chks_lines
chks_direct = chks_direct.set_geometry(dl)
chks_direct.to_pickle(int_data_dir / 'CHKS_direct_lines.pkl')

Finally we have a clean, clear representation of our network of interest. The straight lines between nodes are simply for better visualization; in electrical simulation software, the geographical arrangement of the lines is irrelevant. The length, resistance, reactance, and ampacity of the lines is retained in the Graph database, and so this network can be easily imported into power flow solvers like OpenDSS or pandapower.


In [8]:

# Step 0: Note some important Places of Interest
m = poi.explore(color='green',marker_type='marker', show=False,
                name="Step 0: Note Points of Interest")

# Step 1: Show DRPEP Line Segments
#gdf.explore(m=m,name="Step 1: DRPEP Line Sections",style_kwds=dict(weight=3))
# gdf.explore(m=m,column='randc',cmap='gist_rainbow',legend=False,
#             name="Step 1: DRPEP Line Sections",style_kwds=dict(weight=3))
gdf.explore(m=m,color='blue',legend=False, show=False,
            name="Step 1: DRPEP Line Sections",style_kwds=dict(weight=2))


# Step 2: Find Endpoints
pts.explore(m=m,color='blue',marker_kwds=dict(radius=2),show=False,
               name="Step 2: LineString endpoints")


# Step 3: Blobs
blobs.explore(m=m,color='green',marker_type='circle',
              style_kwds=dict(fillOpacity=0.2),show=False,
              name="Step 3: Combine very-near endpoints into 'blobs'")

# Step 4: Electrical nodes with colors
enodes.explore(m=m,column='randc',cmap='prism',marker_kwds=dict(radius=4),
                legend=False,style_kwds=dict(fillOpacity=1.0),show=False,
                name="Step 4: Infer Electrical Nodes (colored by sub-network)")


# Step 5a: Highlight City Hall/ Karl Storz
chks_lines.explore(m=m,color='red',marker_kwds=dict(radius=7), show=False,
                legend=False,style_kwds=dict(fillOpacity=1.0,weight=3),
                name="Step 5a: Focus on the City Hall/ Karl Storz Subgraph")


# 5b: CHKS direct paths "spider web"
chks_direct.explore(m=m,color='purple',marker_kwds=dict(radius=4),
                legend=False,style_kwds=dict(fillOpacity=1.0,weight=4),
                name="Step 5b: Cleaner Map CHKS Subgraph")


# 5c: CHKS end nodes
chk_enodes.explore(m=m,color='black',marker_kwds=dict(radius=4),
                legend=False,style_kwds=dict(fillOpacity=1.0), 
                name="Step 5c: Electrical Nodes of CHKS Subgraph")

folium.LayerControl().add_to(m)
display(m)